<a href="https://colab.research.google.com/github/RonakArora09/Face-Image-Generator/blob/main/Face_Image_Generator_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FACE IMAGES GENERATOR USING DEEP CONVOLUTIONAL GENERATIVE ADVERSARIAL NETWORK (DCGAN)**

---



Generative Adversarial Network, invented by Ian Goodfellowin in 2014, is one of the coolest technique in Deep Learning. It is an unsupervised algorithm used for Generative modelling. Deep Convolutional GAN (DCGAN) is one of the first branch of GANs which is used for creating new fake images which are in-distinguishable from the original ones.

The technique uses two competing networks, namely **Discriminator** and **Generator**. The Discriminator's job is to classify whether the given image is real or artificial. The Generator task is to generate images to fool Discriminator to believe that the image generated by it is real.

> We are using CelebA dataset available from https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html to train this model. The zip file of the dataset is stored on the drive folder and is extracted in the colab working directory to train the model.







In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/img_align_celeba.zip -d /tmp/celebA

### Parameters used in the model:

**batch_size**  : Number of training images used per model update

**image_dim**   : Shape of the training images

**latent_dim**  : Length of noise input to the generator


In [ ]:
batch_size = 32
image_dim  = (64,64)
latent_dim = 100

## Discriminator Model

In [ ]:
def make_discriminator_model():
    model = keras.models.Sequential()
    
    model.add(keras.layers.Lambda(lambda x: (x-127.5)/127.5))

    model.add(keras.layers.Conv2D(64, (4,4), strides=(2,2), use_bias=False, input_shape = (64,64,3),kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02)))
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Conv2D(128, (4,4), strides=(2,2), use_bias=False,kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Conv2D(256, (4,4), strides=(2,2), use_bias=False,kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Conv2D(512, (4,4), strides=(2,2), use_bias=False,kernel_initializer=tf.keras.initializers.RandomNormal(mean = 0, stddev=0.02)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(0.2))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid, use_bias=False,))

    return model

## Generator Model

In [ ]:
def make_generator_model(latent_dim, noise):

    model = tf.keras.Sequential()

    model.add(keras.layers.Dense(4*4*1024, use_bias=False, input_shape=(latent_dim,)))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())

    model.add(keras.layers.Reshape((4, 4, 1024)))

    model.add(keras.layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())

    model.add(keras.layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())

    model.add(keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.ReLU())

    model.add(keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    model.add(keras.layers.Lambda(lambda x: x*127.5 + 127.5))

    model.noise = noise
    return model

## **DCGAN Model**

In [ ]:
class DCGAN(tf.keras.Model):
    def __init__(self, discriminator = None, generator = None, batch_size = 64, image_dim = (64,64), latent_dim = 128):
        super(DCGAN,self).__init__()

        self.batch_size = batch_size
        self.image_dim  = image_dim
        self.latent_dim = latent_dim
        self.discriminator = discriminator
        self.generator = generator

    def load_images(self, images_path):
        images_ds = tf.keras.preprocessing.image_dataset_from_directory(
            images_path,
            labels = None,
            image_size = self.image_dim,
            batch_size = self.batch_size)
        self.images_ds = images_ds

    def save_current_state(self):
        self.discriminator.save('/content/drive/MyDrive/DCGAN/discriminator')
        self.generator.save('/content/drive/MyDrive/DCGAN/generator')

    def load_models(self):
        discriminator = tf.keras.models.load_model('/content/drive/MyDrive/DCGAN/discriminator')
        generator = tf.keras.models.load_model('/content/drive/MyDrive/DCGAN/generator')
        self.discriminator =  discriminator
        self.generator = generator

    def compile(self, discriminator_optimizer, generator_optimizer, loss_fn, images_path, **kwargs):
        super().compile(**kwargs)

        self.discriminator_optimizer = discriminator_optimizer
        self.generator_optimizer = generator_optimizer
        self.loss_fn = loss_fn
        self.load_images(images_path)

        if(not self.discriminator):
            self.load_models()

    def discriminator_loss(self, real_output, fake_output):
        real_loss = self.loss_fn(np.ones_like(real_output),real_output)
        fake_loss = self.loss_fn(np.zeros_like(fake_output),fake_output)
        discriminator_loss = real_loss + fake_loss
        return discriminator_loss

    def generator_loss(self, fake_output):
        generator_loss = self.loss_fn(np.ones_like(fake_output), fake_output)
        return generator_loss

    def callback(self):
        fake_images = self.generator(self.generator.noise)
        fig = plt.figure(figsize=(16,16))
        plt.axis("off")
        plt.title("These people do not exist")
        for i in range(32):
            sub = fig.add_subplot(8, 4, i + 1)
            sub.axis("off")
            sub.imshow(fake_images.numpy()[i].astype(np.uint8))
        plt.show()
        self.save_current_state()
    

    def train_step(self, real_images):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

            fake_images = self.generator(self.generator.noise)

            real_output = self.discriminator(real_images)
            fake_output = self.discriminator(fake_images)

            disc_loss = self.discriminator_loss(real_output, fake_output)
            gen_loss = self.generator_loss(fake_output)
        
        disc_grad = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)
        self.discriminator_optimizer.apply_gradients(zip(disc_grad, self.discriminator.trainable_variables))

        gen_grad = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        self.generator_optimizer.apply_gradients(zip(gen_grad, self.generator.trainable_variables))

        return disc_loss, gen_loss

    def train(self, epochs = 5):

        iter = 0
        for epoch in range(epochs):
            for real_images in self.images_ds:

                disc_loss, gen_loss = self.train_step(real_images)
                sys.stdout.write(f"\r{iter} >>>> disc_loss: {disc_loss:5e},   gen_loss: {gen_loss:5e}")
                
                if(iter%500 == 0):
                    self.callback()
                iter += 1
            

In [ ]:
noise = np.array([np.random.normal(size=latent_dim) for _ in range(batch_size)])
discriminator = make_discriminator_model()
generator = make_generator_model(latent_dim, noise)
model = DCGAN(discriminator, generator, batch_size=batch_size, image_dim=image_dim, latent_dim=latent_dim)

If no discriminator, generator model given, the model laods the saved models from google drive.

In [ ]:
model = DCGAN()

In [ ]:
discriminator_optimizer = keras.optimizers.RMSprop(learning_rate = 1e-4)
generator_optimizer = keras.optimizers.RMSprop(learning_rate = 1e-4)
loss_fn = tf.keras.losses.BinaryCrossentropy()
images_path = "/tmp/celebA/img_align_celeba/"

model.compile(discriminator_optimizer, generator_optimizer, loss_fn, images_path)

In [ ]:
noise = np.array([np.random.normal(size=latent_dim) for _ in range(batch_size)])
model.generator.noise = noise

In [ ]:
model.train()